<a href="https://colab.research.google.com/github/Ignacio-Ibarra/Approximate-String-Matching/blob/main/Limpieza_Regex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpieza Regex

## Paquetes a utilizar

In [1]:
#Paquetes Básicos
import pandas as pd
import numpy as np
import random
import os
import time
import matplotlib.pyplot as plt
from functools import reduce


#NLP 
import nltk
import re
import string
import unicodedata
from sklearn.feature_extraction.text import TfidfVectorizer
!pip install sparse_dot_topn
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct
 
#Montando Drive en la Session de Google Colab
from google.colab import drive 
drive.mount("/content/drive")
 
#Displaying options
pd.set_option("max_colwidth", 60)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for sparse-dot-topn: filename=sparse_dot_topn-0.3.1-cp37-cp37m-linux_x86_64.whl size=1576536 sha256=a3b213ac7fb3c1eb494ed847d6683351f2fa54bcf24a73b4088934fb9115a589
  Stored in directory: /root/.cache/pip/wheels/62/43/9b/73485cefb3a333e1ec4915cf4db18e964efb597288aa7a45b1
Successfully built sparse-dot-topn
Mounted at /content/drive


## Funcion para leer archivos particionados

In [2]:
import os
from os import listdir

def find_csv_filenames(path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

def spark_csv_to_pandas_df(spark_csv_folder, col_names):
  files_list = find_csv_filenames(spark_csv_folder)
  df = pd.DataFrame()
  for f in files_list:
    path = spark_csv_folder+"/"+f
    temp = pd.read_csv(path, names=col_names)
    df = df.append(temp, ignore_index=True)
  return df

## Funciones de Limpieza

**Lista de Tipos de Entidades** 

Cargamos la base desde [ _Global Legal Entity Identifier_ ](https://www.gleif.org/en/about-lei/introducing-the-legal-entity-identifier-lei) para poder obtener la lista de los tipos de empresa (e.g. "SA", "LLC", "GMBH", etc) y poder borrarlas de los nombres de empresas

In [3]:
glei = pd.read_csv('https://www.gleif.org/content/2-about-lei/7-code-lists/2-iso-20275-entity-legal-forms-code-list/2020-11-19_elf-code-list-v1.3.csv')

In [4]:
ab = ";".join(glei[glei['Abbreviations Local language'].isna()==False]["Abbreviations Local language"].drop_duplicates().values.tolist())

In [5]:
ab = np.unique(np.array(ab.split(";"))).tolist()

In [6]:
abreviaturas = np.unique(np.array([x.upper() for x in ab])).tolist()+["REF"]

In [7]:
#Diccionario con las siglas con puntuación, sin puntuación pero con espacios y sin puntuación ni espacios. 

indices = list(range(1,len(abreviaturas)+1))
 
abrev_dict = dict()
for k,v in zip(abreviaturas, indices): 
  abrev_dict[k]=v

In [8]:
#Definimos función para realizar limpieza

def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')
 
def del_punct_wsp(text): 
    text = text.replace(".","").replace('('," ").replace(")"," ")
    text = text.replace("\M:"," ").replace("M:"," ")
    text = re.sub(r'[!"\#\$%\'\(\)\*\+,\-\./:;<=>\?@\[\\\]\^_`\{\|\}\~]',' ',text) #borra punct y agrega espacio
    text = re.sub(r'\b\d+\b',' ', text)
    text = strip_accents(text)
    return text

def acronyms(text): 
  if ''==text: 
    return ''
  else:
    text = text.upper()
    text = text.split(' ')
    while (text[-1] in abrev_dict) and (len(text)>2):
      text = text[:-1]
      acronyms(' '.join(text))
    return ' '.join(text)
 
def remove_digits(text): 
  splitted = text.split(' ') 
  cleanned = []
  for word in splitted:
    evaluation = [1 if i.isdigit() else 0 for i in word]
    suma = reduce(lambda x,y: x+y, evaluation,0)
    if suma==0:
      cleanned.append(word)
    elif suma<2:
      cleanned.append(word)
    else: 
      word = ''.join([i for i in word if not i.isdigit()])
      cleanned.append(word)
  return " ".join(cleanned)

def strip_spaces(text): 
  return text.upper().lstrip().rstrip()

def remove_within_wsp(text):
  return " ".join(re.findall(r'\b\S+\b', text))

def sepecial_deletions(text, acronyms_list_or_dict): 
  return " ".join([word for word in text.split(" ") if word not in acronyms_list_or_dict])

def pre_processing(text, 
                   punctuation=True, 
                   within_spaces=True, 
                   digits=True, 
                   strip_space=True, 
                   acronyms_at_end=True,
                   special_deletions = None): 
    
    """1) Se borra puntuación, acentos y caracteres específicos como "\M:"
       2) Se borran dígitos
       3) Se remueven espacios en blanco de principio y final
       4) Se borran las siglas al final del texto
       5) Se remueven espacios dentro del texto"""
    if punctuation:
      text = del_punct_wsp(text)
    
    if within_spaces:
      text = remove_within_wsp(text)
    
    if digits:
      text = remove_digits(text)
    
    if strip_space:
      text = strip_spaces(text)

    if special_deletions:
      text = special_deletions(text, special_deletions) 

    if acronyms_at_end:
      text = acronyms(text)
    
    if within_spaces:
      text = remove_within_wsp(text)  

    return text

## Ejecución Limpieza

In [9]:
t0 = time.time()

unicos_sucios_path = '/content/drive/MyDrive/base importadores/outputs/unicos_sucios'
unicos_sucios_pandas = spark_csv_to_pandas_df(unicos_sucios_path, ['unicos_sucios'])
unicos_sucios_pandas = unicos_sucios_pandas.dropna().reset_index(drop=True)
print("Cantidad de Nombres Unicos Sucios: ", len(unicos_sucios_pandas))

#Generamos id en el dataset y borramos nulos
unicos_sucios_pandas = unicos_sucios_pandas.reset_index()
unicos_sucios_pandas.columns = ['id', 'unicos_sucios']

#Hacemos limpieza
unicos_sucios_pandas['first_cleaning'] = unicos_sucios_pandas['unicos_sucios'].apply(lambda x: pre_processing(x,  
                                                                                                            acronyms_at_end=False, 
                                                                                                            special_deletions=False))
t1 = time.time()
print("tiempo en segundo que tardó la primer limpieza. ", t1-t0)

unicos_sucios_pandas.loc[unicos_sucios_pandas.first_cleaning=="",'first_cleaning']="Se borró nombre en limpieza"
n_unicos_limpios = len(unicos_sucios_pandas[unicos_sucios_pandas.first_cleaning!="Se borró nombre en limpieza"]['first_cleaning'].unique())
print("Cantidad de Nombres Unicos Post Regex: ", n_unicos_limpios,"\n")

print("Guadando output en file: sucios_a_first_cleaning.csv\n")
unicos_sucios_pandas.to_csv('/content/drive/MyDrive/base importadores/outputs/sucios_a_first_cleaning/sucios_a_first_cleaning17_06_21.csv', index=False)

Cantidad de Nombres Unicos Sucios:  2312301
tiempo en segundo que tardó la primer limpieza.  229.70869302749634
Cantidad de Nombres Unicos Post Regex:  520666 

Guadando output en file: sucios_a_first_cleaning.csv

